In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c tensorflow-great-barrier-reef

100% 14.2G/14.2G [13:20<00:00, 21.0MB/s]
100% 14.2G/14.2G [13:20<00:00, 19.0MB/s]


In [ ]:
! unzip /content/tensorflow-great-barrier-reef.zip

In [ ]:
! pip install ultralytics
! pip install -U albumentations

In [ ]:
import numpy as np
import os
import pathlib
import pandas as pd
pd.options.mode.chained_assignment = None
import sys
import time
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import cv2 as cv
import albumentations as A
import random
from ultralytics import YOLO
import shutil

In [ ]:
data_df=pd.read_csv('/content/train.csv')
data_df.head()

In [ ]:
if not os.path.exists('/content/datasets'):
    os.makedirs('/content/datasets')

for i in ['train','test','valid']:
  new_path='/content/datasets/'+i
  if not os.path.exists(new_path):
    os.makedirs(new_path)
  for j in ['images','labels']:
    if not os.path.exists(new_path+'/'+j):
      os.makedirs(new_path+'/'+j)

In [ ]:
data_anno=data_df[data_df['annotations']!='[]']
no_boxes=data_anno['annotations'].apply(lambda x:len(x.split('},')))
data_anno['no_boxes']=no_boxes

In [ ]:
# Create imagepath
data_anno['Img_Path']="train_images/"+"video_" + data_df['video_id'].astype(str) + "/" + data_df['video_frame'].astype(str) + ".jpg"
data_tosample=data_anno[data_anno['no_boxes']>1]

In [ ]:
file_names=list(data_tosample['Img_Path'].values)
random.seed(42)
augmented_files = set(random.sample(file_names, int(len(file_names) * 0.85)))
total_bboxes=data_anno['no_boxes'].sum()
total_anno_box=data_anno[data_anno['Img_Path'].isin(list(augmented_files))]['no_boxes'].sum()

In [ ]:
label_dict={'COTS':0}

In [ ]:
def convert_to_yolotxt(b_boxes):
    string=''
    for idx,box in enumerate(b_boxes):
        if idx==0 or len(b_boxes)==1:
            string=string+'0'+' '+str(box[0])+' '+str(box[1])+' '+str(box[2])+' '+str(box[3])
        else:
            string=string+'\n'+'0'+' '+str(box[0])+' '+str(box[1])+' '+str(box[2])+' '+str(box[3])
    return string

In [ ]:
data_df1=data_anno[['Img_Path','no_boxes','annotations']]

In [ ]:
def add_newfilename(old_filename):
    new_filename=old_filename.split('.jpg')[0]+'_aug.jpg'
    return new_filename
def get_img_name(path):
    path=path.split('/')
    return path[-1].split('.jpg')[0]
def write_to_imgfolder(file_path,folder_to_write):
    img_name=file_path.split('/')[-1]
    new_path='datasets/'+folder_to_write+'/images/'
    img_name=img_name.split('.jpg')[0]
    new_path=new_path+img_name+'_aug.jpg'
    return new_path

In [ ]:
def aug_img_bbox(transform,img,b_boxes):
    labels=[]
    for i in range(len(b_boxes)):
        labels.append('COTS')
    transformed = transform(image=img, bboxes=b_boxes, class_labels=labels)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return transformed_image,transformed_bboxes

def write_augmented_files(path,aug_img,aug_bboxes,folder):
    new_path=write_to_imgfolder(path,folder)
    cv.imwrite(new_path,aug_img)
    aug_string=convert_to_yolotxt(aug_bboxes)
    img_name=get_img_name(new_path)
    with open('datasets/'+folder+'/labels/'+img_name+'.txt','w') as file:
        file.write(aug_string)

def write_original_files(path,folder,string):
    shutil.copy(path, 'datasets/'+folder+'/images')
    img_name=get_img_name(path)
    with open('datasets/'+folder+'/labels/'+img_name+'.txt','w') as file:
        file.write(string)


In [ ]:
train=int(total_bboxes*.70)
test=int(total_bboxes*.10)
valid=int(total_bboxes-(train+test))
train_aug=int(total_anno_box*.70)
test_aug=int(total_anno_box*.10)
valid_aug=int(total_anno_box-(train_aug+test_aug))

In [ ]:
def convert_to_yolo_boxes(aug_boxes):
  new_boxes=[]
  for aug_box in aug_boxes:
    xmin  =aug_box[0]
    ymin  =aug_box[1]
    xmax  =aug_box[2]
    ymax  =aug_box[3]
    width =xmax-xmin
    height=ymax-ymin
    new_boxes.append((xmin,ymin,width,height))
  return new_boxes


In [ ]:
for path,no_boxes,anno in data_df1.values:
    img=cv.imread(path)
    lisst=eval(anno)
    b_boxes=[]
    temp_boxes=[]
    for i in lisst:
        x1=i['x']
        y1=i['y']
        width=i['width']
        height=i['height']
        x_center=(x1+(width/2))/img.shape[1]
        y_center=(y1+(height/2))/img.shape[0]
        x_max=x1+i['width']
        y_max=y1+i['height']
        aug_temp=[x1/img.shape[1],y1/img.shape[0],x_max/img.shape[1],y_max/img.shape[0]]
        temp1=[idx for idx,i in enumerate(aug_temp) if i>1]
        if len(temp1)>0:
            for idx in temp1:
                aug_temp[idx]=1.0
        box_width=width/img.shape[1]
        box_height=height/img.shape[0]
        temp=[x_center,y_center,box_width,box_height]
        b_boxes.append(temp)
        temp_boxes.append(aug_temp)
    string=convert_to_yolotxt(b_boxes)
    if train>0:
        train=train-len(b_boxes)
        write_original_files(path,'train',string)
    elif valid>0:
        valid=valid-len(b_boxes)
        write_original_files(path,'valid',string)
    else :
        test=test-len(b_boxes)
        write_original_files(path,'test',string)

    if path in list(augmented_files):
        num=random.randint(0,2)
        if num==0:
            transform = A.Compose([A.HorizontalFlip(p=1)], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels']))
            aug_img,aug_bboxes=aug_img_bbox(transform,img,temp_boxes)
            aug_bboxes=convert_to_yolo_boxes(aug_bboxes)
            if train_aug>0:
                train_aug=train_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'train')
            elif valid_aug>0:
                valid_aug=valid_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'valid')
            else :
                test_aug=test_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'test')
        elif num==1:
            transform = A.Compose([A.BBoxSafeRandomCrop(p=1)], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels']))
            aug_img,aug_bboxes=aug_img_bbox(transform,img,temp_boxes)
            aug_bboxes=convert_to_yolo_boxes(aug_bboxes)
            if train_aug>0:
                train_aug=train_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'train')
            elif valid_aug>0:
                valid_aug=valid_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'valid')
            else :
                test_aug=test_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'test')
        else:
            transform = A.Compose([A.RandomRotate90(p=1)], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels']))
            aug_img,aug_bboxes=aug_img_bbox(transform,img,temp_boxes)
            aug_bboxes=convert_to_yolo_boxes(aug_bboxes)
            if train_aug>0:
                train_aug=train_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'train')
            elif valid_aug>0:
                valid_aug=valid_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'valid')
            else :
                test_aug=test_aug-len(aug_bboxes)
                write_augmented_files(path,aug_img,aug_bboxes,'test')

In [ ]:
model = YOLO('yolov8m.pt')
model.train(
   epochs=100,
   imgsz=[540,960],
   data='data.yaml'
)

Ultralytics YOLOv8.0.57 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=100, patience=50, batch=16, imgsz=[540, 960], save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=

In [ ]:
img=cv.imread('/content/datasets/train/images/1005.jpg')
with open('/content/datasets/train/labels/1005.txt','r') as file:
  for line in file:
    coords=list(map(float,line.split(' ')[1:]))
    x_center=int(coords[0]*img.shape[1])
    y_center=int(coords[1]*img.shape[0])
    b_width=int(coords[2]*img.shape[1])
    b_height=int(coords[3]*img.shape[0])
    x1=int(x_center-(b_width/2))
    y1=int(y_center-(b_height/2))
    x2=x1+b_width
    y2=y1+b_height
    cv.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv.imwrite('/content/output.jpg',img)

True

In [ ]:
model = YOLO('/content/runs/detect/train/weights/best.pt')  # load a custom model

# Validate the model
# metrics = model.val()  # no arguments needed, dataset and settings remembered
# metrics.box.map    # map50-95
# metrics.box.map50  # map50
# metrics.box.map75  # map75
# metrics.box.maps

In [ ]:
result=model.predict(source='/content/datasets/test/images',
                     save=True,
                     conf=0.4,
                     imgsz=[864,1536],
                     )


image 1/540 /content/datasets/test/images/10619.jpg: 864x1536 (no detections), 24.2ms
image 2/540 /content/datasets/test/images/10620.jpg: 864x1536 (no detections), 22.9ms
image 3/540 /content/datasets/test/images/10621.jpg: 864x1536 (no detections), 22.7ms
image 4/540 /content/datasets/test/images/10622.jpg: 864x1536 (no detections), 23.1ms
image 5/540 /content/datasets/test/images/10623.jpg: 864x1536 (no detections), 23.1ms
image 6/540 /content/datasets/test/images/10624.jpg: 864x1536 (no detections), 22.9ms
image 7/540 /content/datasets/test/images/10625.jpg: 864x1536 (no detections), 22.9ms
image 8/540 /content/datasets/test/images/10626.jpg: 864x1536 (no detections), 23.5ms
image 9/540 /content/datasets/test/images/10627.jpg: 864x1536 (no detections), 22.7ms
image 10/540 /content/datasets/test/images/10628.jpg: 864x1536 (no detections), 22.9ms
image 11/540 /content/datasets/test/images/10629.jpg: 864x1536 (no detections), 24.8ms
image 12/540 /content/datasets/test/images/10630.jp

In [ ]:
count=0
for res in result:
  count+=len(res.boxes.xyxy)

In [ ]:
count

529

In [ ]:
!zip -r /content/file.zip /content/Folder_To_Zip

In [ ]:
!zip -r /content/yolov8_output.zip /content/runs/detect/predict

In [ ]:
for i in result:
  print(i.box.xyxy)

NameError: ignored